In [6]:
import os
import wave
import numpy as np
from scipy.io import wavfile


def extract_first_13_seconds_from_wav(input_file):
    with wave.open(input_file, 'rb') as wav_file:
        num_frames = wav_file.getnframes()
        sample_width = wav_file.getsampwidth()
        num_channels = wav_file.getnchannels()
        frame_rate = wav_file.getframerate()
        total_duration = num_frames / float(frame_rate)

        target_duration = 13.0  
        percentage = target_duration / total_duration

        start_frame = 0
        end_frame = int(num_frames * percentage)

        if end_frame < 0:
            end_frame = 0
        if end_frame >= num_frames:
            end_frame = num_frames - 1

        output_directory = os.path.dirname(input_file)
        output_file = os.path.join(output_directory, 'output.wav')

        with wave.open(output_file, 'wb') as output_wav:
            output_wav.setnchannels(num_channels)
            output_wav.setsampwidth(sample_width)
            output_wav.setframerate(frame_rate)

            output_wav.setnframes(end_frame - start_frame)
            wav_file.setpos(start_frame)

            output_wav.writeframes(wav_file.readframes(end_frame - start_frame))

    # Downsampling begins here
    # Read the output .wav file
    sample_rate, data = wavfile.read(output_file)

    # Half the sample rate
    new_rate = sample_rate // 2

    # Interleave the data if it's stereo
    if len(data.shape) > 1:
        data = np.mean(data, axis=1)

    # Downsample the data
    downsampled = data[::2]

    # Write the new .wav file
    compressed_output_file = os.path.join(output_directory, 'compressed_output.wav')
    wavfile.write(compressed_output_file, new_rate, downsampled.astype(np.int16))


In [7]:
input_file = 'airi_0.wav'

extract_first_13_seconds_from_wav(input_file)